In [78]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import re

In [79]:
pd.set_option('display.max_columns',40)
options = webdriver.ChromeOptions()
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument('--disable-extensions')
options.add_argument("--disable-gpu")

df=pd.DataFrame()
Compound_Name=[]
Attibute_Name=[]
Custome_Value=[]
tille_index=[]
timeout=10
url = f'https://www.nhathuocankhang.com/hoat-chat?substancekey='

In [80]:
service = Service('/ChromeDriver/chromedriver_win32/chromedriver.exe')
driver = webdriver.Chrome(service=service,options=options)
def get_Url(url,key_page):
    li_text=[]
    while True:
        try:
            url_page =url+key_page
            driver.get(url_page)
            div_content=driver.find_element(By.XPATH,'/html/body/section/div[2]')
            ul_tags=div_content.find_element(By.TAG_NAME,'ul')
            li_tags=ul_tags.find_elements(By.TAG_NAME,'li')
            break
        except TimeoutException:
            driver.refresh()
    for li in li_tags:
        li_text.append(li.text)
    return li_text

In [81]:
def get_Value_Href(div_content):
    list_href=[]
    a_tags=[]
    ul_tags=div_content.find_element(By.TAG_NAME,'ul')
    li_tags=ul_tags.find_elements(By.TAG_NAME,'li')
    for li in li_tags:
        a_tag=li.find_element(By.TAG_NAME,'a')
        a_tags.append(a_tag)
    for a in a_tags:
        href=a.get_attribute('href')
        list_href.append(href)
    return list_href



In [82]:
def get_Text_Li_Tags(div_content):
    li_text=[]
    ul_tags=div_content.find_element(By.TAG_NAME,'ul')
    li_tags=ul_tags.find_elements(By.TAG_NAME,'li')
    for li in li_tags:
        li_text.append(li.text)
    return li_text

In [83]:
def get_Title_And_Content(list_li_text,key_page):
    compound_name=[]
    compound_title=[]
    compound_content=[]
    try:
        j=0
        while j<len(list_li_text):
            li_text_=list_li_text[j]
            try:
                time.sleep(2)
                try:
                    items = driver.find_element(By.XPATH,'/html/body/section/div[2]/ul/li[{}]/a'.format(j+1)).click()
                    time.sleep(2)
                    content_wrapper=driver.find_element(By.XPATH,'/html/body/section/div[1]')
                    articles=content_wrapper.find_elements(By.TAG_NAME,'article')
                    # print('Len tag Articles của | ',li_text_,' | ',len(articles))
                    j+=1
                except Exception as e:
                    print('Error :\n',e)
                    driver.get(url+key_page)
                    time.sleep(2)
                    pass
                if (len(articles)>1):
                    for i in range(1,len(articles)):
                        title=articles[i].find_element(By.CLASS_NAME,'title').text
                        compound_title.append(title)
                        content=articles[i].find_element(By.CLASS_NAME,'content').text
                        compound_content.append(content)
                    compound_name[len(compound_name):] = [li_text_] * (len(articles)-1)
                    driver.back()
                else:
                    driver.back()
            except Exception as e:
                print('Error :\n',e)
                driver.get(url+key_page)
                time.sleep(2)
        
    except Exception as e:
        print('Error :\n',e)
        driver.get(url+key_page)
        time.sleep(2)
    return compound_name,compound_title,compound_content

In [84]:
def get_All_Href(all_page):
    arr_href=[]
    for i,key in enumerate(all_page):
        try:
            li_tags=get_Url(url,key)
            time.sleep(2)
            substance=driver.find_element(By.XPATH,'/html/body/section/div[2]/div')
            list_tag_a=substance.find_elements(By.TAG_NAME,'a')
        except:
            pass
        print('List li tag:',len(list_tag_a))
        try:
            for i,a in enumerate(list_tag_a):
                print(a.text)
                list_href=[]
                a.click()
                time.sleep(1)
                print('Click vào page:',i+1)
                div_content=driver.find_element(By.XPATH,'/html/body/section/div[2]')
                list_href=get_Value_Href(div_content)
                for h in list_href:
                    arr_href.append(h)
                time.sleep(1)
        except:
            print('Không có page')
            div_content=driver.find_element(By.XPATH,'/html/body/section/div[2]')
            list_href=get_Value_Href(div_content)
            for h in list_href:
                arr_href.append(h)
    return arr_href

In [ ]:
List_All_Href=get_All_Href('abcdefghiklmnopqrstuvwxyz')
for href in List_All_Href:
    driver.get(href)
    try:
        time.sleep(2)
        try:
            ul_head=driver.find_element(By.XPATH,'/html/body/section/ul')
            lis=ul_head.find_elements(By.TAG_NAME,'li')
            li_text=lis[2].text
            content_wrapper=driver.find_element(By.XPATH,'/html/body/section/div[1]')
            articles=content_wrapper.find_elements(By.TAG_NAME,'article')
            # print('Len tag Articles của | ',li_text_,' | ',len(articles))
        except Exception as e:
            print('Error :\n',e)
            driver.get(href)
            time.sleep(2)
            pass
        if (len(articles)>1):
            for i in range(1,len(articles)):
                title=articles[i].find_element(By.CLASS_NAME,'title').text
                Attibute_Name.append(title)
                content=articles[i].find_element(By.CLASS_NAME,'content').text
                Custome_Value.append(content)
            Compound_Name[len(Compound_Name):] = [li_text] * (len(articles)-1)
            driver.back()
        else:
            driver.back()
    except Exception as e:
        print('Error :\n',e)
        driver.get(href)
        time.sleep(2)
    

In [86]:
print('Độ dài của list name:',len(Compound_Name))
print('Độ dài của list Attribute:',len(Attibute_Name))
print('Độ dài của list content:',len(Custome_Value))

Độ dài của list name: 10264
Độ dài của list Attribute: 10264
Độ dài của list content: 10264


In [87]:
df['Compound_Name']=Compound_Name
df['Attibute_Name']=Attibute_Name
df['Content_Value']=Custome_Value

In [ ]:

driver.get(List_All_Href[0])
lis=ul_head.find_elements(By.TAG_NAME,'li')
li_text=lis[1].text
li_text

In [93]:
df.to_csv('/FRT/16.Side_Effect/7.Crawl_data_ankhang/all.csv')

In [71]:
ul_head=driver.find_element(By.XPATH,'/html/body/section/ul')
lis=ul_head.find_elements(By.TAG_NAME,'li')
li_text=lis[2]

In [ ]:
Compound_Name[0].text


In [76]:
li_text.text

'Allantoin'